In [1]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import torch
from uuid import uuid4
import os


/Users/jeremyhsieh/Documents/USC Course/Penkova Lab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    print("MPS backend is available! 🚀")
else:
    print("MPS backend is not available. 😢")

MPS backend is available! 🚀


In [3]:
# check default path of model storage
from transformers.utils import default_cache_path
print(default_cache_path)

/Users/jeremyhsieh/.cache/huggingface/hub


In [4]:
def path_check(path):
    if not os.path.exists(path):
        os.mkdir(path)
    return path

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder = path_check("../model")
)

text = "Hello, how are you?"
embedding_vector = embeddings.embed_query(text)
print(f"Embedding vector size: {len(embedding_vector)}")


Embedding vector size: 384


In [14]:
vector_store = Chroma(
    collection_name="leukemia_info",
    embedding_function=embeddings,
    persist_directory=path_check("../chroma_langchain_db"),  # Where to save data locally, remove if not necessary
)

In [ ]:
# vector_store._collection.delete(ids=['1','2','3','4','5','6'])

Delete of nonexisting embedding ID: 1
Delete of nonexisting embedding ID: 2
Delete of nonexisting embedding ID: 3
Delete of nonexisting embedding ID: 4
Delete of nonexisting embedding ID: 5
Delete of nonexisting embedding ID: 6
Delete of nonexisting embedding ID: 1
Delete of nonexisting embedding ID: 2
Delete of nonexisting embedding ID: 3
Delete of nonexisting embedding ID: 4
Delete of nonexisting embedding ID: 5
Delete of nonexisting embedding ID: 6


In [15]:
document_1 = Document(
    page_content = '''
        Conjunctiva involvement, although not a common presentation of leukemias, occurs most often in
        patients with lymphocytic leukaemias.6.Cellular involvement is found at all levels of the substantia propria and can be diffuse or patchy, tending to concentrate along blood vessels.7 Comma-shaped
        venial abnormalities (cork screw vessels similar to
        those found in sickle cell disease) have also been reported. These are believed to be secondary to hyperviscosity.8 ALL has been reported to present as conjunctival lymphoma in adults.9 Lei et al described an unusual occurrence of bilateral conjunctival tumours
        in a 25-year-old woman. This was the first sign of relapse of ALL.
    ''',
    metadata={"ophthalmic_involvement": "Conjunctiva", "disease": "leukemia"},
    id=1,
)

document_2 = Document(
    page_content = '''
        The cornea is an avascular structure and therefore is not commonly involved in leukaemia, especially in the form of direct invasion by leukaemic cells. Allen and Straatsma's series reported no corneal involvement beyond limbal infiltration.7
        Sterile ring ulcers with iritis and pannus have been reported in leukaemias.11 Keratitis can occur secondary to immunosuppression or graft-versus-host disease (GVHD). Keratitis in a patient with GVHD can cause severe thining of cornea, threatening corneal perforation.12 Corneal involvement is also seen when corneal epithelial changes result secondary to chemotherapy.13 These changes include thinning irregularity, faulty maturation, and keratinisation. Peripheral corneal ulceration has also been reported in a patient with leukaemia and herpes zoster ophthalmicus.14
        Scleral infiltration is usually an autopsy finding and occurs in acute leukaemia. These cells are most often found in the episclera in a perivascular pattern.7
        ''',
    metadata={"ophthalmic_involvement": "Cornea and sclera", "disease": "leukemia"},
    id=2,
)

document_3 = Document(
    page_content ='''
        Clinically evident infiltration of the iris by leukaemic cells is not common. 
        It occurs with the involvement of choroid and ciliary body. 
        Clinically, it is characterised by a change in iris colour, and a pseudohypopyon, which is grey/yellow in colour.15 
        On histopathological examination, the iris may show diffuse involvement, especially at the root and sphincter.7 
        The intra-ocular pressure (IOP) can be high enough to cause signs and symptoms of acute glaucoma with normal depth of anterior chamber.16 
        It is postulated that the raised IOP is probably due to infiltration of the trabecular meshwork.17 In children, spontaneous hyphaema is also a presentation of leukaemia.15 
        Usually clinically apparent involvement of the iris and anterior segment occurs with ALL.17, 18 It may also occur less commonly with CLL19 and myeloid leukaemias.15 
        Extramedullary relapse of acute leukaemias may masquerade as hypopyon uveitis.20 
        Primary relapse of acute leukaemia in the anterior segment is not uncommon.20, 21, 22, 23 
        Leukaemias have been identified as the cause of uveitis in 5% of paediatric uveitis cases.23 
        Ocular involvement is unusual in acute non-lymphoblastic leukaemia, 
        however, one case was reported where an infant who had evidence of active central nervous system disease showed infiltration of the anterior chamber during therapy. 
        Treatment was with topical corticosteroids, chemotherapy, and bilateral ocular radiotherapy.24 Any ophthalmic manifestation in children with leukaemia should be detected and treated early. 
        Radiotherapy is warranted in infiltration of the anterior chamber. The presence of ocular or CNS involvement indicates poor prognosis in acute childhood leukaemia.
        ''',
    metadata={"ophthalmic_involvement": "Iris and anterior segment", "disease": "leukemia"},
    id=3,
)

document_4 = Document(
    page_content = '''
Choroid shows leukaemic infiltration most consistently on histopathological examination, though clinically the retina is most commonly involved in leukaemia.7.
        The involvement of choroid by leukaemic cells tends to be perivascular, and may be patchy or diffuse.7 
        Choroid may be thickened to many times normal at the posterior pole. 
        The overlying retinal pigment epithelium (RPE) may show secondary alterations, including atrophy and hypertrophy. 
        There may be secondary photoreceptor cell loss, drusen formation or serous detachment. 
        Clinically, choroidal involvement presents as a serous retinal detachment, which is generally shallow, and located at the posterior pole. 
        These detachments have been reported in CLL25, ALL26, 27, CML28 and AML.29
    ''',
    metadata={"ophthalmic_involvement": "Choroid", "disease": "leukemia"},
    id=4,
)

document_5 = Document(
    page_content = '''
        The retina is involved in leukaemia more often than any other ocular tissue. 
        It is estimated that up to 69% of all patients with leukaemia show fundus changes at some point in the course of their disease, 
        although at that time no effective treatments were present.30 The early manifestations (because of haematological disturbance)
        are venous dilatation and tortuosity.31 Haemorrhages may occur in all levels of the retina, usually in the posterior pole, 
        and may extend into the vitreous. They may be round or flame shaped, and often has a white component. The white area consists 
        of leukaemic cells and debris, platelet-fibrin aggregates, or septic emboli. A similar clinical picture can be seen in severe 
        anaemia (Figure 1), thrombocytopenia, and hyperviscosity. Histopathological examination shows discrete and diffuse haemorrhage 
        and leukaemic infiltration.7 The haemorrhages and infiltrates are found at all levels of the retina, but especially in the inner 
        layers with focal destruction. The infiltrates and aggregates of leukaemic cells are usually but not always seen with surrounding 
        haemorrhage.32 Large leukaemic infiltrates can cause total retinal detachment that may present as an isolated relapse.33 Smaller 
        infiltrates tend to be perivascular. Subretinal infiltration in leukaemia has been referred as subretinal hypopyon.34 Cotton wool 
        spots can be seen and are probably due to ischaemia from anaemia, hyperviscosity, or leukaemic infiltration.
        ''',
    metadata={"ophthalmic_involvement": "Retina", "disease": "leukemia"},
    id=5,
)

document_6 = Document(
    page_content = '''
        Immune Response: When your body fights a cold, it triggers an immune response that can lead to inflammation in various parts of your body, including your eyes. 
        Viral Conjunctivitis (Pink Eye): A cold can sometimes cause viral conjunctivitis, also known as pink eye, which can lead to red, itchy, and watery eyes. 
        Nasal Congestion: Nasal congestion can also contribute to watery eyes, as the nasal passages and eyes are connected. 
        ''',
    metadata={"ophthalmic_involvement": "Redness and Watery Eyes", "disease": "cold"},
    id=6,
)


In [16]:
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['6d7fda27-d05c-4d23-9567-1f51ec0de7b6',
 'e7785e1d-9fd5-485c-a8f3-558d894aeb69',
 '6952bbd0-4a6c-4ba4-834a-fb1f7dcff0cd',
 '190a58b8-e4c5-4a25-a827-211afacef337',
 'da09aa61-8041-4732-93b6-438b887d2602',
 'f7a93fec-2606-4712-be7b-77c917868ffa']

In [17]:
results = vector_store.similarity_search(
    "Does Retinopathy associated with leukemia typically occur in acute disease.",
    k=3,
    # filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 
        The retina is involved in leukaemia more often than any other ocular tissue. 
        It is estimated that up to 69% of all patients with leukaemia show fundus changes at some point in the course of their disease, 
        although at that time no effective treatments were present.30 The early manifestations (because of haematological disturbance)
        are venous dilatation and tortuosity.31 Haemorrhages may occur in all levels of the retina, usually in the posterior pole, 
        and may extend into the vitreous. They may be round or flame shaped, and often has a white component. The white area consists 
        of leukaemic cells and debris, platelet-fibrin aggregates, or septic emboli. A similar clinical picture can be seen in severe 
        anaemia (Figure 1), thrombocytopenia, and hyperviscosity. Histopathological examination shows discrete and diffuse haemorrhage 
        and leukaemic infiltration.7 The haemorrhages and infiltrates are found at all levels of the 

In [18]:
class CustomRetrievalChain():
    def __init__(self, llm, vector_store, memory, top_k: int = 3):
        self.llm = llm
        self.vector_store = vector_store
        self.memory = memory
        self.top_k = top_k
        self.system_template = '''summarize info and use info to answer user's question'''

    def run(self) -> str:
        while True:
            user_input = input("\nUser: ")

            if user_input.lower() == 'exit':
                self.memory.clear()  
                print("Good Bye~")
                break

            related_docs = self.vector_store.similarity_search(query=user_input, k=self.top_k)
            retrieved_texts = "\n".join([doc.page_content for doc in related_docs])
            
            full_input = f"User input:{user_input}\nInfo:\n{retrieved_texts}"
            
            prompt = ChatPromptTemplate.from_messages([
                SystemMessagePromptTemplate.from_template(self.system_template),
                MessagesPlaceholder(variable_name="history"),
                HumanMessagePromptTemplate.from_template("{formatted_input}")
                ]
            )

            conversation = ConversationChain(
                memory=self.memory,
                prompt=prompt,
                llm=self.llm,
                verbose=False,
                input_key="formatted_input" 
            )

            response = conversation.predict(formatted_input=full_input)
            print(f"\nChatbot: {response}")


llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini", api_key = "YOUR_API_KEY")
memory = ConversationBufferMemory(return_messages=True)
vector_store = Chroma(
    collection_name="leukemia_info",
    embedding_function=embeddings,
    persist_directory=path_check("../chroma_langchain_db"),  # Where to save data locally
)

retrieval_chain = CustomRetrievalChain(llm, vector_store, memory, 3)
retrieval_chain.run()


/var/folders/vy/qg2xs8zs2wvcchmgz5ttrvg40000gn/T/ipykernel_43895/435293806.py:43: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/var/folders/vy/qg2xs8zs2wvcchmgz5ttrvg40000gn/T/ipykernel_43895/435293806.py:30: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(



Chatbot: Retinopathy associated with leukemia is more common in patients with the disease, especially during acute phases. Approximately 69% of leukemia patients exhibit changes in the retina at some point, with early signs including venous dilatation and tortuosity. Hemorrhages can occur at various levels of the retina, particularly in the posterior pole, and may extend into the vitreous. These hemorrhages are often accompanied by leukaemic cells and debris, and can lead to significant retinal issues, including total retinal detachment in severe cases. Thus, retinopathy related to leukemia is typically observed in acute disease settings.
Good Bye~
